## RANDOM FOREST 

Data Loading from train_for_model.csv and holdout

70-15-15 split with preprocessing

Random Forest training using MultiOutputRegressor

Evaluation on:

Validation set

Internal test set

External holdout set

Saving:

Model .pkl

Input/output scalers

Metrics CSVs

Predictions CSVs

Plots (scatter, residual) - need to research

### Starting with 25 percent training data to cehck the working of the Random forest and faster execution for comparisons

- The models are saved 
- Need further RMSE values in better format for float and also need the understanding on the predictions with the csv files to compare manually

Checking the voltage_rise_time_pulse value desciptions

Add this in the feature understanding - better there 

In [ ]:
target_columns = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]



In [13]:
print(y['voltage_rise_time_pulse1'].describe())

count   1.206e+05
mean    1.228e-08
std     2.095e-08
min     4.708e-09
25%     7.225e-09
50%     1.277e-08
75%     1.581e-08
max     5.045e-06
Name: voltage_rise_time_pulse1, dtype: float64


In [12]:
# Assuming y is a DataFrame containing these columns
print(y['voltage_rise_time_pulse2'].describe())

count    1.206e+05
mean    -2.551e-06
std      1.402e-05
min     -8.089e-05
25%      6.670e-09
50%      1.060e-08
75%      1.415e-08
max      1.823e-06
Name: voltage_rise_time_pulse2, dtype: float64


In [14]:
print(y['voltage_fall_time_pulse1'].describe())

count   1.206e+05
mean    2.354e-06
std     6.238e-06
min     3.978e-09
25%     1.218e-08
50%     1.734e-08
75%     2.239e-08
max     2.102e-05
Name: voltage_fall_time_pulse1, dtype: float64


Again execution on the same fast mode and all the saved files but need the complete execution

In [ ]:
# src/train/random_forest_train.py
# Usage: $env:PYTHONPATH="."; python Classical_Models\RF\random_forest_train.py

import os
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# === CONFIGURATION ===
FAST_MODE = True
TRAIN_FILE = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\data\processed\train_for_model.csv"
HOLDOUT_FILE = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\data\processed\merged_test_with_features.csv"
MODEL_DIR = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\random_forest\testing"
os.makedirs(MODEL_DIR, exist_ok=True)

TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID']

# === 1. Load & Sample Training Data ===
df = pd.read_csv(TRAIN_FILE)

df.dropna(subset=TARGET_COLUMNS, inplace=True)
if FAST_MODE:
    df = df.sample(frac=0.25, random_state=42)


X = df.drop(columns=DROP_COLUMNS + TARGET_COLUMNS)
y = df[TARGET_COLUMNS]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

# === 2. Scaling ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

joblib.dump(scaler_X, os.path.join(MODEL_DIR, "input_scaler.pkl"))
joblib.dump(scaler_y, os.path.join(MODEL_DIR, "output_scaler.pkl"))

# === 3. Train Model ===
n_estimators = 30 if FAST_MODE else 100
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=n_estimators, random_state=42, n_jobs=-1))
start_time = time.time()
model.fit(X_train_scaled, y_train_scaled)
print(f"Training completed in {time.time() - start_time:.2f} seconds")

# === 4. Evaluation Function ===
def evaluate(model, X_scaled, y_scaled, label, save_name):
    y_pred_scaled = model.predict(X_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')

    print(f"\n{label} Evaluation:")
    for i, col in enumerate(TARGET_COLUMNS):
        print(f"{col:<30} RMSE: {rmse[i]:.6f}  R²: {r2[i]:.4f}")

    # Save actual vs predicted
    df_result = pd.DataFrame({
        f"{col}_actual": y_true[:, i] for i, col in enumerate(TARGET_COLUMNS)
    })
    for i, col in enumerate(TARGET_COLUMNS):
        df_result[f"{col}_predicted"] = y_pred[:, i]

    df_result.to_csv(os.path.join(MODEL_DIR, save_name), index=False)
    print(f"Saved predictions to: {save_name}")

# === 5. Evaluate All Splits ===
evaluate(model, X_val_scaled, y_val_scaled, "Validation", "validation_predictions.csv")
evaluate(model, X_test_scaled, y_test_scaled, "Internal Test", "internal_test_predictions.csv")

# === 6. Evaluate on Holdout Test ===
df_holdout = pd.read_csv(HOLDOUT_FILE)
df_holdout.dropna(subset=TARGET_COLUMNS, inplace=True)
X_holdout = df_holdout.drop(columns=DROP_COLUMNS + TARGET_COLUMNS)
y_holdout = df_holdout[TARGET_COLUMNS]

X_holdout_scaled = scaler_X.transform(X_holdout)
y_holdout_scaled = scaler_y.transform(y_holdout)
evaluate(model, X_holdout_scaled, y_holdout_scaled, "External Holdout", "holdout_predictions.csv")

# === 7. Save Final Model ===
joblib.dump(model, os.path.join(MODEL_DIR, "model.pkl"))
print(f"\nModel and scalers saved to: {MODEL_DIR}")


Training completed in 130.86 seconds

Validation Evaluation:
voltage_rise_time_pulse1       RMSE: 0.000000  R²: 0.0122
voltage_rise_time_pulse2       RMSE: 0.000000  R²: 0.9999
voltage_fall_time_pulse1       RMSE: 0.000000  R²: 1.0000
voltage_fall_time_pulse2       RMSE: 0.000000  R²: 0.9999
current_rise_time_pulse1       RMSE: 0.000001  R²: 0.9945
current_rise_time_pulse2       RMSE: 0.000000  R²: 0.9342
current_fall_time_pulse1       RMSE: 0.000005  R²: 0.7723
current_fall_time_pulse2       RMSE: 0.000000  R²: 0.5838
overshoot_pulse_1              RMSE: 0.369711  R²: 0.9998
overshoot_pulse_2              RMSE: 0.127537  R²: 1.0000
undershoot_pulse_1             RMSE: 3.486054  R²: 0.9819
undershoot_pulse_2             RMSE: 9.392370  R²: 0.9460
ringing_frequency_MHz          RMSE: 1.151400  R²: 0.9993
Saved predictions to: validation_predictions.csv

Internal Test Evaluation:
voltage_rise_time_pulse1       RMSE: 0.000000  R²: 0.8993
voltage_rise_time_pulse2       RMSE: 0.000000  R²: 

Training completed in 130.86 seconds

Validation Evaluation:
voltage_rise_time_pulse1       RMSE: 0.000000  R²: 0.0122
voltage_rise_time_pulse2       RMSE: 0.000000  R²: 0.9999
voltage_fall_time_pulse1       RMSE: 0.000000  R²: 1.0000
voltage_fall_time_pulse2       RMSE: 0.000000  R²: 0.9999
current_rise_time_pulse1       RMSE: 0.000001  R²: 0.9945
current_rise_time_pulse2       RMSE: 0.000000  R²: 0.9342
current_fall_time_pulse1       RMSE: 0.000005  R²: 0.7723
current_fall_time_pulse2       RMSE: 0.000000  R²: 0.5838
overshoot_pulse_1              RMSE: 0.369711  R²: 0.9998
overshoot_pulse_2              RMSE: 0.127537  R²: 1.0000
undershoot_pulse_1             RMSE: 3.486054  R²: 0.9819
undershoot_pulse_2             RMSE: 9.392370  R²: 0.9460
ringing_frequency_MHz          RMSE: 1.151400  R²: 0.9993

Saved predictions to: validation_predictions.csv

Internal Test Evaluation:
voltage_rise_time_pulse1       RMSE: 0.000000  R²: 0.8993
voltage_rise_time_pulse2       RMSE: 0.000000  R²: 0.9999
voltage_fall_time_pulse1       RMSE: 0.000000  R²: 1.0000
voltage_fall_time_pulse2       RMSE: 0.000000  R²: 0.9999
current_rise_time_pulse1       RMSE: 0.000001  R²: 0.9969
current_rise_time_pulse2       RMSE: 0.000000  R²: 0.9386
...
ringing_frequency_MHz          RMSE: 5.093005  R²: 0.8357
Saved predictions to: holdout_predictions.csv

Model and scalers saved to: C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\random_forest\testing
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

To save the actual and preduicted evaluations of R2 and RMSE as it is important to compare of the validation, holout etc 

In [15]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# === Save directory
MODEL_DIR = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\random_forest"
os.makedirs(MODEL_DIR, exist_ok=True)

def evaluate(model, X_scaled, y_scaled, label, save_name):
    y_pred_scaled = model.predict(X_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    # === Compute RMSE and R²
    rmse = np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')

    # === Pretty print results
    print(f"\n{label} Evaluation:")
    print(f"{'Target Output':<35} {'RMSE':>15} {'R² Score':>12}")
    print("-" * 64)
    for i, col in enumerate(TARGET_COLUMNS):
        rmse_sci = f"{rmse[i]:.2E}"
        r2_val = f"{r2[i]:.4f}"
        print(f"{col:<35} {rmse_sci:>15} {r2_val:>12}")

    # === Save predictions (actual vs predicted)
    df_result = pd.DataFrame({
        f"{col}_actual": y_true[:, i] for i, col in enumerate(TARGET_COLUMNS)
    })
    for i, col in enumerate(TARGET_COLUMNS):
        df_result[f"{col}_predicted"] = y_pred[:, i]

    predictions_path = os.path.join(MODEL_DIR, save_name)
    df_result.to_csv(predictions_path, index=False)
    print(f"Predictions saved to: {predictions_path}")

    # === Save RMSE + R² metrics
    df_metrics = pd.DataFrame({
        "Target": TARGET_COLUMNS,
        "RMSE": [f"{val:.2E}" for val in rmse],
        "R2_Score": [round(val, 4) for val in r2]
    })

    metrics_filename = save_name.replace(".csv", "_metrics.csv")
    metrics_path = os.path.join(MODEL_DIR, metrics_filename)
    df_metrics.to_csv(metrics_path, index=False)
    print(f"Metrics saved to: {metrics_path}")


Saving the files

In [16]:
evaluate(model, X_val_scaled, y_val_scaled, "Validation", "validation_predictions.csv")
evaluate(model, X_test_scaled, y_test_scaled, "Internal Test", "internal_test_predictions.csv")
evaluate(model, X_holdout_scaled, y_holdout_scaled, "External Holdout", "holdout_predictions.csv")



Validation Evaluation:
Target Output                                  RMSE     R² Score
----------------------------------------------------------------
voltage_rise_time_pulse1                   3.74E-08       0.0122
voltage_rise_time_pulse2                   1.55E-07       0.9999
voltage_fall_time_pulse1                   3.51E-09       1.0000
voltage_fall_time_pulse2                   5.42E-08       0.9999
current_rise_time_pulse1                   8.34E-07       0.9945
current_rise_time_pulse2                   1.66E-07       0.9342
current_fall_time_pulse1                   5.45E-06       0.7723
current_fall_time_pulse2                   2.63E-08       0.5838
overshoot_pulse_1                          3.70E-01       0.9998
overshoot_pulse_2                          1.28E-01       1.0000
undershoot_pulse_1                         3.49E+00       0.9819
undershoot_pulse_2                         9.39E+00       0.9460
ringing_frequency_MHz                      1.15E+00       0.9993
P


Validation Evaluation:
Target Output                                  RMSE     R² Score
----------------------------------------------------------------
voltage_rise_time_pulse1                   3.74E-08       0.0122
voltage_rise_time_pulse2                   1.55E-07       0.9999
voltage_fall_time_pulse1                   3.51E-09       1.0000
voltage_fall_time_pulse2                   5.42E-08       0.9999
current_rise_time_pulse1                   8.34E-07       0.9945
current_rise_time_pulse2                   1.66E-07       0.9342
current_fall_time_pulse1                   5.45E-06       0.7723
current_fall_time_pulse2                   2.63E-08       0.5838
overshoot_pulse_1                          3.70E-01       0.9998
overshoot_pulse_2                          1.28E-01       1.0000
undershoot_pulse_1                         3.49E+00       0.9819
undershoot_pulse_2                         9.39E+00       0.9460
ringing_frequency_MHz                      1.15E+00       0.9993
Predictions saved to: C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\random_forest\validation_predictions.csv
Metrics saved to: C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\random_forest\validation_predictions_metrics.csv

Internal Test Evaluation:
Target Output                                  RMSE     R² Score
----------------------------------------------------------------
voltage_rise_time_pulse1                   1.40E-09       0.8993
voltage_rise_time_pulse2                   1.50E-07       0.9999
...
undershoot_pulse_2                         1.41E+00       0.9859
ringing_frequency_MHz                      5.09E+00       0.8357
Predictions saved to: C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\random_forest\holdout_predictions.csv
Metrics saved to: C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\random_forest\holdout_predictions_metrics.csv
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...